In [47]:
import json
import os
import pandas as pd
import requests
from tqdm import tqdm
import psycopg2
import math

In [63]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()


In [20]:
word = []
#definition = []
local_word = []
examples = []
examples_local = []
parent = "./y_lookup/en_ru/"
for file in os.listdir(parent):
    with open(os.path.join(parent,file), "r") as f: 
        data = json.load(f)
        for definition in data['def']:
            word_current =  definition['text']
            for translation in definition['tr']: 
                word.append(word_current)
                local_word.append(translation['text'])
                ex_en = []
                ex_rus = []
                if 'ex' in translation:
                    for exmpl in translation['ex']:
                        #print(exmpl)
                        ex_en.append(exmpl['text'])
                        ex_rus.append(exmpl['tr'][0]['text'])
                examples.append(ex_en)
                examples_local.append(ex_rus)
                #print(translation)
                #print()
    

In [2]:
def parse_from_yandex_lookup(directory, print_output = False):
    word = []
    local_word = []
    examples = []
    examples_local = []
    
    for file in os.listdir(directory):
        if print_output == True: print(file)
        with open(os.path.join(directory,file), "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    data = pd.DataFrame(list(zip(word,local_word,examples, examples_local)),columns =['word', 'local_word', 'examples','local_examples'])
    return  data


In [69]:
def get_word_forms(word):
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not req_res:
        print(word, " is not in db")
        return []
    else:
        #print(a[0])
        word_id = req_res[0]
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    word_forms = []
    for word in cursor:
        #print(word[0])
        word_forms.append(word[0])
    return word_forms
get_word_forms("book")


['booker', 'booked', 'booking', 'books']

In [3]:
def parse_json(path, word, local_word,examples,examples_local):
    try:
        with open(path, "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    except:
        pass

In [72]:
def parse_from_yandex_lookup_vs_wordforms(directory):
    word = []
    local_word = []
    examples = []
    examples_local = []
    words = pd.read_csv("test_words.csv", header = None)
    for word_i in words[0].items():
        print (word_i[1])
        file = word_i[1] + ".json"
        parse_json(os.path.join(directory,file),word, local_word,examples,examples_local)
        wordforms = get_word_forms(word_i[1].lower())
        for wf in wordforms:
            print(">>>", wf)
            file = wf + ".json"
            parse_json(os.path.join(directory,file),word, local_word,examples,examples_local)
        
    data = pd.DataFrame(list(zip(word,local_word,examples, examples_local)),columns =['word', 'local_word', 'examples','local_examples'])
    return  data
enru_wv = parse_from_yandex_lookup_vs_wordforms("./y_lookup/en_ru/")

word
>>> wording
>>> worded
>>> words
the
>>> thebed
>>> thest
>>> the bed
>>> thebest
>>> thes
and
>>> anding
>>> ands
>>> ood
>>> andren
>>> anded
>>> ander
that
>>> that led
>>> thats
>>> those
for
>>> forring
>>> fored
>>> fors
with
>>> withim
>>> withing
>>> withs
but
>>> buts
you
>>> youed
>>> yous
>>> youta
>>> youx
>>> youing
>>> youger
this
>>> thises
>>> this led
>>> these
not
>>> not bed
>>> notest
>>> notting
>>> nots
have
>>> have led
>>> haveta
>>> haveing
>>> had
>>> haved
>>> haves
>>> having
>>> havest
letter
>>> lettered
>>> letterd
>>> lettering
>>> letters
>>> lettera
plate
>>> plating
>>> plated
>>> plates
treat
>>> treats
>>> treated
>>> treaten
>>> treating
>>> treatest
dinner
>>> dinners
necessary
>>> necessarie
go
>>> gota
>>> gos
>>> gone
>>> went
>>> goed
>>> goes
>>> goren
>>> going
secret
>>> secrets
>>> secret
>>> secreter
table
>>> tables
>>> tabled
>>> tabling
>>> tabler
create
>>> creating
>>> creates
>>> createing
>>> created
warm
>>> warmest
>>> warme

>>> befores
begin
>>> begins
>>> began
>>> beginning
>>> beginer
>>> begining
>>> beginnest
>>> begun
>>> begin
behind
>>> behinds
below
>>> below
>>> belowed
>>> belows
best
>>> bester
>>> bests
>>> best best
>>> besting
>>> bestest
>>> bested
synchrophasotron
synchrophasotron  is not in db


In [75]:
enru_wv.to_csv("yandex_lookup_en_ru_with_wordforms.csv")

In [74]:
enru_wv.head(30)

,word,local_word,examples,local_examples
0,word,слово,"[last word, old english word, new word in art,...","[последнее слово, старое английское слово, нов..."
1,word,известие,[],[]
2,word,слова,[],[]
3,word,говорить,[],[]
4,word,сформулировать,[],[]
5,word,текстовый,[word processing document],[текстовый документ]
6,word,словесный,[word game],[словесная игра]
7,wording,формулировка,[exact wording],[точная формулировка]
8,wording,редакция,[new wording],[новая редакция]
9,wording,текст,[wording of the declaration],[текст декларации]


In [76]:
en_fr_wv = parse_from_yandex_lookup_vs_wordforms("./y_lookup/en_fr/")

word
>>> wording
>>> worded
>>> words
the
>>> thebed
>>> thest
>>> the bed
>>> thebest
>>> thes
and
>>> anding
>>> ands
>>> ood
>>> andren
>>> anded
>>> ander
that
>>> that led
>>> thats
>>> those
for
>>> forring
>>> fored
>>> fors
with
>>> withim
>>> withing
>>> withs
but
>>> buts
you
>>> youed
>>> yous
>>> youta
>>> youx
>>> youing
>>> youger
this
>>> thises
>>> this led
>>> these
not
>>> not bed
>>> notest
>>> notting
>>> nots
have
>>> have led
>>> haveta
>>> haveing
>>> had
>>> haved
>>> haves
>>> having
>>> havest
letter
>>> lettered
>>> letterd
>>> lettering
>>> letters
>>> lettera
plate
>>> plating
>>> plated
>>> plates
treat
>>> treats
>>> treated
>>> treaten
>>> treating
>>> treatest
dinner
>>> dinners
necessary
>>> necessarie
go
>>> gota
>>> gos
>>> gone
>>> went
>>> goed
>>> goes
>>> goren
>>> going
secret
>>> secrets
>>> secret
>>> secreter
table
>>> tables
>>> tabled
>>> tabling
>>> tabler
create
>>> creating
>>> creates
>>> createing
>>> created
warm
>>> warmest
>>> warme

>>> befores
begin
>>> begins
>>> began
>>> beginning
>>> beginer
>>> begining
>>> beginnest
>>> begun
>>> begin
behind
>>> behinds
below
>>> below
>>> belowed
>>> belows
best
>>> bester
>>> bests
>>> best best
>>> besting
>>> bestest
>>> bested
synchrophasotron
synchrophasotron  is not in db


In [78]:
en_fr_wv.to_csv("yandex_lookup_en_fr_with_wordforms.csv")

In [21]:
len(word), len(local_word), len(examples), len(examples_local)

(676, 676, 676, 676)

In [35]:
enru_ = parse_from_yandex_lookup("./y_lookup/en_ru/")
enru.head()

,word,local_word,examples,local_examples
0,for,для,"[for the bank, for last month, for one year, f...","[для банка, за последний месяц, на один год, н..."
1,for,ради,[for god],[ради бога]
2,for,в течение,[],[]
3,for,в отношении,[],[]
4,for,для целей,[],[]


In [ ]:
enru

In [23]:
enfr__whole_list = parse_from_yandex_lookup("./y_lookup/en_fr_whole_list/", print_output=True)
enfr__whole_list.head()

for.json
before.json
adverb.json
during.json
airport.json
exit.json
drum.json
empty.json
dressed.json
cafe.json
flight.json
begin.json
fact.json
dream.json
alarm.json
along.json
follow.json
warm.json
collide.json
excellent.json
break.json
examination.json
tiger.json
engineer.json
lion.json
even.json
field.json
background.json
easily.json
write.json
ravioli.json
apologise.json
fire.json
fly.json
beaver.json
duck.json
coincide.json
exactly.json
anything.json
exhaustively.json
apoplexy.json
coconut.json
not.json
international.json
synchrophasotron.json
best.json
parent.json
ocean.json
coincidental.json
buzzer.json
century.json
euro.json
dynamite.json
buy.json
afraid.json
peremptory.json
member.json
childhood.json
go.json
plate.json
fish.json
farmer.json
stand.json
drawing.json
explain.json
typhus.json
precedent.json
advertisement.json
issue.json
fifty.json
thousand.json
aged.json
file.json
pen.json
fan.json
pasta.json
hot.json
that.json
engine.json
advanced.json
be.json
but.json
and.json


,word,local_word,examples,local_examples
0,for,pour,[],[]
1,before,avant,[],[]
2,before,auparavant,[],[]
3,adverb,adverbe,[],[]
4,during,lors,[],[]


In [27]:
list(enfr__whole_list['word']).index("nobody")

ValueError: 'nobody' is not in list

In [39]:
enru.to_csv("yandex_lookup_en_ru_lemmas_only.csv")

In [21]:
enfr__whole_list.to_csv("yandex_lookup_en_fr_lemmas_only_WHOLE_LIST.csv")

In [44]:
url_y = "http://192.168.122.13:31436/GetTranslate"
h_y = {"apiVersion": "1.0.0", "text": "гайки и болты", "langPair": {"source": "ru", "target": "fr"}}
#h_y = {"apiVersion": "1.0.0", "text": "Landing to the mars", "langPair": "en-ru"}
r = requests.post(url = url_y, json=h_y)
data = r.json()
data

{'translate': 'écrous et boulons'}

In [41]:
def y_translate(text, from_lang, to_lang):
    url_y = "http://192.168.122.13:31436/GetTranslate"
    h_y = {"apiVersion": "1.0.0", "text": text, "langPair": {"source": from_lang, "target": to_lang}}
    r = requests.post(url = url_y, json=h_y)
    data = r.json()
    return data['translate']
y_translate("привет уеба", "ru", "fr")

'salut, connard.'

In [14]:
enru_small = parse_from_yandex_lookup("./y_lookup/en_ru/")
len(enru_small)

1190

19

1410

In [4]:
enru_big = parse_from_yandex_lookup("./y_lookup/yandex_lookup_big_en_ru/")
print(len(enru_big))
enru_big.head()

1410


,word,local_word,examples,local_examples
0,for,для,"[for the bank, for last month, for one year, f...","[для банка, за последний месяц, на один год, н..."
1,for,ради,[for god],[ради бога]
2,for,в течение,[],[]
3,for,в отношении,[],[]
4,for,для целей,[],[]


In [5]:
enru_big.to_csv("./lookup results/yandex_lookup_en_ru_lemmas_only_WHOLE_LIST.csv")

In [58]:
def naive_translation(en_ru_dataframe, from_lang,to_lang):
    word = []
    local_word = []
    example = []
    local_example = []
    for element_ind in tqdm(range(len(en_ru_dataframe))):
        el_list = list(en_ru_dataframe.iloc[element_ind])
        word.append(el_list[1])
        print(el_list)
        #print(el_list[2])
        
        try:
            tr_local_word = y_translate(el_list[2],from_lang,to_lang)
            print(el_list[2], "ok")
        except:
            tr_local_word = "no_translation"
            print("failed", el_list[1])
        local_word.append(tr_local_word)
        ex = ''
        loc_ex = ''
        try:
            for example_i in el_list[3].split("|"):
                if example_i:
                    ex += example_i + '|'
                    try:
                        ex_tr = y_translate(example_i,"en",to_lang)
                        print(example_i, "ok")
                    except:
                        ex_tr = "no_translation"
                        print("failed", example_i)
                    loc_ex+= ex_tr + '|'
        except:
            print("no examples")
        ex = ex[:-1]
        loc_ex = loc_ex[:-1]
        example.append(ex)
        local_example.append(loc_ex)
        if element_ind % 10 == 0: 
            data = pd.DataFrame(list(zip(word,local_word,example, local_example)),columns =['word', 'local_word', 'examples','local_examples'])
            data.to_csv("/Users/nigula/LL/adjust_unigr_dict/y_translate/systran_enrufr_naive/"+str(element_ind)+".csv")
        
        #if element_ind > 10: break
    data = pd.DataFrame(list(zip(word,local_word,example, local_example)),columns =['word', 'local_word', 'examples','local_examples'])
    return data

In [ ]:
df_enru_whole = naive_translation(enru_big,"ru","fr")
df_enru_whole.head()

In [18]:
df_enru_whole.to_csv("naive_transaltion_en_ru_fr_LEMMAS_WHOLE_LIST.csv")

# SYSTRAN EN RU TO FR NAIVE

In [30]:
df_systran_en_ru = pd.read_csv("systran_en_ru.csv")

In [32]:
df_systran_en_ru.head(12)

,Unnamed: 0,word,local_word,examples,local_examples,definitions
0,0,word,слово,key word|english word|first word|good word|,ключевое слово | английское слово | первое сло...,speech | expression | term |
1,1,and,и,NaN,NaN,|
2,2,and,а,NaN,NaN,but | while |
3,3,and,и,He thinks of you day and night|,Он думает о вас день и ночь |,NaN
4,4,and,но,It's an embarrassing joke and not an insult|,"Это неуместная шутка, а не оскорбление |",NaN
5,5,and,а,This is a squid and not an octopus|,"Это кальмар, а не осьминог |",NaN
6,6,that,так,NaN,NaN,as | so | as well as | how | so that | both |
7,7,that,то,NaN,NaN,|
8,8,that,который / кто / тот который,I know the guy that sold you the disk|,"Я знаю парня, который продал тебе диск |",NaN
9,9,that,тот,On that evening I met Natalie|Do you see those...,В тот вечер я встретил Натали | Видишь те две ...,NaN


In [59]:
df_systran_en_ru_fr = naive_translation(df_systran_en_ru,"ru","fr")





  0%|          | 0/1358 [00:00<?, ?it/s]

[0, 'word', 'слово', 'key word|english word|first word|good word|', 'ключевое слово | английское слово | первое слово | хорошее слово | ', 'speech | expression | term | ']
слово ok
key word ok
english word ok
first word ok
good word ok






  0%|          | 1/1358 [00:02<1:07:03,  2.96s/it]

[1, 'and', 'и', nan, nan, ' | ']






  0%|          | 2/1358 [00:03<48:32,  2.15s/it]  

...

наилучший ok
no examples
[1356, 'well', 'зажиточный', nan, nan, 'affluent | wealthy | well-off | prosperous | well-to-do | ']






100%|█████████▉| 1357/1358 [20:30<00:00,  1.35it/s]

зажиточный ok
no examples
[1357, 'well', 'колодезный', nan, nan, ' | ']






100%|██████████| 1358/1358 [20:30<00:00,  1.63it/s]

колодезный ok
no examples


In [61]:
list(df_systran_en_ru['local_word'])

['слово',
 'и',
 'а',
 'и',
 'но',
 'а',
 'так',
 'то',
 'который / кто / тот который',
 'тот',
 'что',
 'чтобы',
 'для',
 'на',
 'за',
 'в',
 'по',
 'к',
 'потому что / ибо',
 'с',
 'без',
 'но',
 'однако',
 'а',
 'но',
 'а',
 'тем не менее',
 'однако',
 'как / кроме как',
 'чтобы не',
 'вы',
 'ты',
 'этот',
 'это',
 'нельзя',
 'мало',
 'недостаточно',
 'совсем',
 'плохо',
 'невозможно',
 'иметь',
 'быть',
 'получить',
 'обладать',
 'оказать',
 'мочь',
 'быть',
 'попалить',
 'придтись',
 'вынудить',
 'приходиться',
 'следовать',
 'потребоваться',
 'обязать',
 'иметь',
 'письмо',
 'буква',
 'символ',
 'пластина',
 'плита',
 'тарелка',
 'пластинка',
 'лист',
 'табличка',
 'листы',
 'посуда',
 'доска',
 'угощение',
 'удовольствие',
 'лакомство',
 'лечить',
 'рассматривать',
 'относиться',
 'обращаться',
 'вылечить',
 'рассматриваться',
 'угостить',
 'обрабатывать',
 'лечиться',
 'diners',
 'ужин',
 'обед',
 'нужный',
 'обязательный',
 'необходимый',
 'неотъемлемый',
 'должный',
 'идти',


In [63]:
df_systran_en_ru_fr['rus_word'] = list(df_systran_en_ru['local_word'])

In [66]:
df_systran_en_ru_fr = df_systran_en_ru_fr.reindex(columns = ['word','rus_word', 'local_word','examples','local_examples'])

In [67]:
df_systran_en_ru_fr.head(100)

,word,rus_word,local_word,examples,local_examples
0,word,слово,mot,key word|english word|first word|good word,le mot-clé|mot anglais|premier mot|bonne parole
1,and,и,et,,
2,and,а,et,,
3,and,и,et,He thinks of you day and night,Il pense à toi jour et nuit
4,and,но,mais,It's an embarrassing joke and not an insult,C'est embarrassant blague et pas une insulte
5,and,а,et,This is a squid and not an octopus,C'est un calmar et pas un poulpe
6,that,так,donc,,
7,that,то,ce,,
8,that,который / кто / тот который,qui / qui / celui qui,I know the guy that sold you the disk,Je connais le gars qui vous a vendu le disque
9,that,тот,le,On that evening I met Natalie|Do you see those...,"Sur la soirée, j'ai rencontré Natalie|Voyez-vo..."


In [68]:
df_systran_en_ru_fr.to_csv("enrufr_naive_translation_systran_lookup_whole_list.csv")